In [1]:
from sentence_transformers import SentenceTransformer
import polars as pl

# this function encodes the given text using the sentence transformer and truncates/pads as necessary
def encode(text):
    enc = [x.item() for x in list(tokenizer.encode(text))]
    return enc
    
inputs = 'jobs_all_filtered.csv'
output = 'processed.parquet'

tokenizer = SentenceTransformer("msmarco-distilbert-dot-v5")

In [2]:
# read in data
df = pl.scan_csv(inputs)
df = df.drop_nulls(subset='description')

In [3]:
df.collect()

job_id,company_id,title,description,max_salary,med_salary,min_salary,pay_period,formatted_work_type,location,state_abbr,applies,original_listed_time,remote_allowed,views,job_posting_url,application_url,application_type,expiry,closed_time,formatted_experience_level,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,scraped,industry_id,industry_name,skill_abr,skill_name,company_name,company_description,company_size,company_state,company_country,company_city,company_zipcode,company_address,company_url,company_industry,company_employee_count,company_follower_count,company_time_recorded
i64,i64,str,str,f64,f64,f64,str,str,str,str,i64,f64,i64,i64,str,str,str,f64,f64,str,str,f64,str,i64,str,str,str,i64,i64,str,str,str,str,str,i64,str,str,str,str,str,str,str,i64,i64,i64
3757744847,3594609,"""Virtual Assita…","""Company Descri…",null,6000.0,null,"""MONTHLY""","""Full-time""","""United States""","""US""",24,1.7000e12,1,87,"""https://www.li…",null,"""ComplexOnsiteA…",1.7000e12,null,null,null,1.7000e12,null,0,"""FULL_TIME""","""USD""","""BASE_SALARY""",1699061274,80,"""Advertising Se…","""ADM""","""Administrative…","""Transformed De…","""Transformed De…",null,"""California""","""US""","""Vista""","""92081""","""0""","""https://www.li…","""Advertising Se…",3,46,1699061274
3748851001,2662,"""Administrative…","""JOB TITLE: Adm…",7828.0,6710.0,5592.0,"""YEARLY""","""Full-time""","""San Francisco,…","""CA""",2,1.7000e12,null,17,"""https://www.li…","""https://career…","""OffsiteApply""",1.7000e12,null,"""Entry level""",null,1.7000e12,null,0,"""FULL_TIME""","""USD""","""BASE_SALARY""",1698994437,75,"""Government Adm…","""ADM""","""Administrative…","""Judicial Counc…","""The Judicial C…",4,"""California""","""US""","""San Francisco""","""0""","""455 Golden Gat…","""https://www.li…","""Administration…",798,7277,1698994437
3755585509,17902,"""Assembler - 1s…","""About JLG, An …",null,10000.0,null,"""YEARLY""","""Full-time""","""Greencastle, P…","""PA""",null,1.7000e12,null,1,"""https://www.li…","""https://oshkos…","""OffsiteApply""",1.7000e12,null,"""Entry level""",null,1.7000e12,"""oshkoshcorpora…",0,"""FULL_TIME""","""USD""","""BASE_SALARY""",1699079710,55,"""Machinery Manu…","""MGMT""","""Management""","""JLG Industries…","""Official JLG I…",5,"""PA""","""US""","""McConnellsburg…","""17233-9533""","""1 JLG Drive""","""https://www.li…","""Machinery""",2377,81821,1692849011
3749373359,207350,"""Aircraft Inter…","""Aircraft Inter…",null,10000.0,null,"""YEARLY""","""Full-time""","""Oklahoma City,…","""OK""",null,1.7000e12,null,1,"""https://www.li…",null,"""SimpleOnsiteAp…",1.7000e12,null,"""Entry level""",null,1.7000e12,"""aarcorp.taleo.…",1,"""FULL_TIME""","""USD""","""BASE_SALARY""",1699060460,94,"""Airlines and A…","""ENG""","""Engineering""","""AAR""","""AAR is an inde…",6,"""IL""","""US""","""Wood Dale""","""60191""","""1100 North Woo…","""https://www.li…","""Airlines and A…",4622,56807,1699052209
3749371543,207350,"""Avionics Crew …","""Avionics Crew …",null,10000.0,null,"""YEARLY""","""Full-time""","""Oklahoma City,…","""OK""",null,1.7000e12,null,null,"""https://www.li…",null,"""SimpleOnsiteAp…",1.7000e12,null,"""Mid-Senior lev…",null,1.7000e12,"""aarcorp.taleo.…",0,"""FULL_TIME""","""USD""","""BASE_SALARY""",1699083491,94,"""Airlines and A…","""MGMT""","""Management""","""AAR""","""AAR is an inde…",6,"""IL""","""US""","""Wood Dale""","""60191""","""1100 North Woo…","""https://www.li…","""Airlines and A…",4622,56807,1699052209
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
3697353978,9308035,"""Distinguished …","""Crossover is t…",null,800000.0,null,"""YEARLY""","""Full-time""","""Phoenix, AZ""","""AZ""",6,1.6900e12,1,45,"""https://www.li…","""https://www.cr…","""ComplexOnsiteA…",1.7000e12,null,"""Executive""",null,1.6900e12,"""www.crossover.…",1,"""FULL_TIME""","""USD""","""BASE_SALARY""",1,4,"""Software Devel…","""ART""","""Art/Creative""","""Crossover ""","""Cr

In [4]:
new_df = df.with_columns(
    [pl.col("description").map_elements(encode, return_dtype=pl.List(pl.Float64)).alias('Encoding')]
)

In [5]:
collected = new_df.collect()
collected

job_id,company_id,title,description,max_salary,med_salary,min_salary,pay_period,formatted_work_type,location,state_abbr,applies,original_listed_time,remote_allowed,views,job_posting_url,application_url,application_type,expiry,closed_time,formatted_experience_level,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,scraped,industry_id,industry_name,skill_abr,skill_name,company_name,company_description,company_size,company_state,company_country,company_city,company_zipcode,company_address,company_url,company_industry,company_employee_count,company_follower_count,company_time_recorded,Encoding
i64,i64,str,str,f64,f64,f64,str,str,str,str,i64,f64,i64,i64,str,str,str,f64,f64,str,str,f64,str,i64,str,str,str,i64,i64,str,str,str,str,str,i64,str,str,str,str,str,str,str,i64,i64,i64,list[f64]
3757744847,3594609,"""Virtual Assita…","""Company Descri…",null,6000.0,null,"""MONTHLY""","""Full-time""","""United States""","""US""",24,1.7000e12,1,87,"""https://www.li…",null,"""ComplexOnsiteA…",1.7000e12,null,null,null,1.7000e12,null,0,"""FULL_TIME""","""USD""","""BASE_SALARY""",1699061274,80,"""Advertising Se…","""ADM""","""Administrative…","""Transformed De…","""Transformed De…",null,"""California""","""US""","""Vista""","""92081""","""0""","""https://www.li…","""Advertising Se…",3,46,1699061274,"[-0.073653, 0.040756, … -0.102961]"
3748851001,2662,"""Administrative…","""JOB TITLE: Adm…",7828.0,6710.0,5592.0,"""YEARLY""","""Full-time""","""San Francisco,…","""CA""",2,1.7000e12,null,17,"""https://www.li…","""https://career…","""OffsiteApply""",1.7000e12,null,"""Entry level""",null,1.7000e12,null,0,"""FULL_TIME""","""USD""","""BASE_SALARY""",1698994437,75,"""Government Adm…","""ADM""","""Administrative…","""Judicial Counc…","""The Judicial C…",4,"""California""","""US""","""San Francisco""","""0""","""455 Golden Gat…","""https://www.li…","""Administration…",798,7277,1698994437,"[-0.350865, 0.16235, … -0.09886]"
3755585509,17902,"""Assembler - 1s…","""About JLG, An …",null,10000.0,null,"""YEARLY""","""Full-time""","""Greencastle, P…","""PA""",null,1.7000e12,null,1,"""https://www.li…","""https://oshkos…","""OffsiteApply""",1.7000e12,null,"""Entry level""",null,1.7000e12,"""oshkoshcorpora…",0,"""FULL_TIME""","""USD""","""BASE_SALARY""",1699079710,55,"""Machinery Manu…","""MGMT""","""Management""","""JLG Industries…","""Official JLG I…",5,"""PA""","""US""","""McConnellsburg…","""17233-9533""","""1 JLG Drive""","""https://www.li…","""Machinery""",2377,81821,1692849011,"[0.089385, 0.626603, … -0.480057]"
3749373359,207350,"""Aircraft Inter…","""Aircraft Inter…",null,10000.0,null,"""YEARLY""","""Full-time""","""Oklahoma City,…","""OK""",null,1.7000e12,null,1,"""https://www.li…",null,"""SimpleOnsiteAp…",1.7000e12,null,"""Entry level""",null,1.7000e12,"""aarcorp.taleo.…",1,"""FULL_TIME""","""USD""","""BASE_SALARY""",1699060460,94,"""Airlines and A…","""ENG""","""Engineering""","""AAR""","""AAR is an inde…",6,"""IL""","""US""","""Wood Dale""","""60191""","""1100 North Woo…","""https://www.li…","""Airlines and A…",4622,56807,1699052209,"[-0.463898, 0.581835, … -0.146492]"
3749371543,207350,"""Avionics Crew …","""Avionics Crew …",null,10000.0,null,"""YEARLY""","""Full-time""","""Oklahoma City,…","""OK""",null,1.7000e12,null,null,"""https://www.li…",null,"""SimpleOnsiteAp…",1.7000e12,null,"""Mid-Senior lev…",null,1.7000e12,"""aarcorp.taleo.…",0,"""FULL_TIME""","""USD""","""BASE_SALARY""",1699083491,94,"""Airlines and A…","""MGMT""","""Management""","""AAR""","""AAR is an inde…",6,"""IL""","""US""","""Wood Dale""","""60191""","""1100 North Woo…","""https://www.li…","""Airlines and A…",4622,56807,1699052209,"[-0.228599, 0.753888, … -0.093517]"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
3697353978,9308035,"""Distinguished …","""Crossover is t…",null,800000.0,null,"""YEARLY""","""Full-time""","""Phoenix, AZ""","""AZ""",6,1.6900e12,1,45,"""https://www.li…","""https://www.cr…","""C

In [6]:
collected.write_parquet(output)